# Naive Gaussian Elimination

Three operations applied to a system of linear equations which yield an equivalent system:
1. Swap one equation for another
2. Add or subtract a multiple of one euqation from another
3. Multiply an equation by a non-zero constant

Suppose we have,
$$ x + y = 3 $$
$$ 3x - 4y = 2 $$
by subtracting 3 times the first equation from the second equation
$$ x + y = 3$$
$$ - 7y = - y $$
We can then backsolve to a solution $(x, y) = (2, 1)$.


Foreshadowing Gaussian elimination,

$$
\left[
\begin{array}{cc|c}
 1 & 1 & 3\\
 3 & -4 & 2 \\
\end{array}
\right]
$$
Becomes,
$$
\left[
\begin{array}{cc|c}
 1 & 1 & 3\\
 0 & -7 & -7 \\
\end{array}
\right]
$$
after subtracting 3 times the first row from the first row.

$$
\left[
\begin{array}{cccc|c}
 a_{11} & a_{12} & \cdots & a_{1n} & b_1 \\
 a_{21} & a_{22} & \cdots & a_{2n} & b_2 \\
 \vdots & \vdots & \cdots & \vdots & \vdots \\
 a_{n1} & a_{n2} & \cdots & a_{nn} & b_n \\
\end{array}
\right]
$$

**elimination step:** To carry out this step, we need to put zeros in the lower triangular area of the augmented matrix - use row operations to put a zero in each location below the diagonal.
The location sof the main diagonal are then given by $a_{j+1,j}, a_{j+2,j},...,a_{nj}$.
```
for j in range(n-1):
    eliminate column j
```